<a href="https://colab.research.google.com/github/Jin0331/TA/blob/master/Bigdata_2020/Bigdata_Lab_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **빅데이터 개론 Lab 11 - 교차분석, 데이터베이스 연동**

참고자료 : https://www.notion.so/TA-2689a38b5289413a82671d3956fea103

- - -




### **<교차분석>**

* 두 범주형 변수 간에 연관관계(association)가 있는지의 여부를 판단하고자 하는 경우 교차표를 작성하여 변수들 간의 관계를 분석한다.

 - 이를 교차분석 혹은 χ2(chi-square) 검정(test)이라고 한다. 교차분석은 두 변수의 빈도표를 교차 시킨다는 의미로 해석되며, 교차분석에 사용되는 검정 통계량이 ``χ2 -분포를 따르므로 χ2 - 검정``으로 부른다.

 - 교차표(cross tabulation), 분할표(contingency table): 각 범주형 변수에 대한 빈도표를 행과 열로 결합시켜 놓은 형태이다. 일반적으로 행에는 설명(독립)변수에 해당 하는 변수를 할당하고, 열에는 반응(종속)변수를 할당한다. 


In [ ]:
install.packages(c("tidyverse", "data.table", "gmodels"))
library(tidyverse)
library(data.table)
library(gmodels)
options(repr.plot.width=13, repr.plot.height=13)

#### **A. 예제 1**

---


1. **HR(Human Resource, 인사관리)_연습데이터([링크](https://www.dropbox.com/sh/xx1w2syi768kfu0/AACZgxgo1fcxyDMgv9U-iTz8a?dl=0))**

* 변수 설명

```
satisfaction_level : 직무 만족도
last_evaluation : 마지막 평가점수
number_project : 진행 프로젝트 수
average_monthly_hours : 월평균 근무시간
time_spend_company : 근속년수
work_accident : 사건사고 여부(0: 없음, 1: 있음, 범주형)
left : 이직 여부(0: 잔류, 1: 이직, 범주형)
promotion_last_5years: 최근 5년간 승진여부(0: 승진 x, 1: 승진, 범주형)
sales : 부서(범주형)
salary : 임금 수준(범주형)
```


In [ ]:
hr_DF <- fread("https://raw.githubusercontent.com/Jin0331/TA/master/data/HR_comma_sep.csv", sep = ",") %>% as_tibble()

In [ ]:
hr_DF %>% str()

In [ ]:
hr_DF %>% summary()

* 범주형 변수 확인

In [ ]:
hr_DF$sales %>% unique()

In [ ]:
hr_DF$salary %>% unique()

In [ ]:
hr_DF$Work_accident %>% unique()

In [ ]:
hr_DF$left %>% unique()

In [ ]:
hr_DF$promotion_last_5years %>% unique()

* mutate를 이용한 데이터 타입 변경(int or chr ---> factor)

In [ ]:
hr_DF <- hr_DF %>% mutate_at(`.vars` = c("sales", "salary", "Work_accident", "left", "promotion_last_5years"), `.funs` = as.factor)
hr_DF %>% str()

* left(이직여부)와 salary(연봉수준)의 연관성 검사

In [ ]:
gmodels::CrossTable(hr_DF$left, hr_DF$salary, 
                    chisq = T, # 카이제곱 검정
                    expected = T, # 기대값(E) 표시
                    prop.r = F, prop.c = F) # 각 셀의 비율표시(F)

* left(이직여부)와 promotion_last_5years(5년간 승진여부)의 연관성 검사

In [ ]:
gmodels::CrossTable(hr_DF$left, hr_DF$promotion_last_5years, 
                    chisq = T, # 카이제곱 검정
                    expected = T, # 기대값(E) 표시
                    prop.r = F, prop.c = F) # 각 셀의 비율표시(F)

* left(이직여부)와 Work_accident(사건사고 여부)의 연관성 검사

In [ ]:
gmodels::CrossTable(hr_DF$left, hr_DF$Work_accident, 
                    chisq = T, # 카이제곱 검정
                    expected = T, # 기대값(E) 표시
                    prop.r = F, prop.c = F) # 각 셀의 비율표시(F)

* left(이직여부)와 sales(부서)의 연관성 검사

In [ ]:
gmodels::CrossTable(hr_DF$left, hr_DF$sales, 
                    chisq = T, # 카이제곱 검정
                    expected = T, # 기대값(E) 표시
                    prop.r = F, prop.c = F) # 각 셀의 비율표시(F)

* promotion_last_5years(5년간 승진여부)와 Work_accident(사건사고 여부)의 연관성 검사

In [ ]:
gmodels::CrossTable(hr_DF$promotion_last_5years, hr_DF$Work_accident, 
                    chisq = T, # 카이제곱 검정
                    expected = T, # 기대값(E) 표시
                    prop.r = F, prop.c = F) # 각 셀의 비율표시(F)

* Work_accident(사건사고 여부)와 sales(부서)의 연관성 검사

In [ ]:
gmodels::CrossTable(hr_DF$Work_accident, hr_DF$sales, 
                    chisq = T, # 카이제곱 검정
                    expected = T, # 기대값(E) 표시
                    prop.r = F, prop.c = F) # 각 셀의 비율표시(F)

* salary(연봉)과 sales(부서)의 연관성 검사


In [ ]:
gmodels::CrossTable(hr_DF$salary, hr_DF$sales, 
                    chisq = T, # 카이제곱 검정
                    expected = T, # 기대값(E) 표시
                    prop.r = F, prop.c = F) # 각 셀의 비율표시(F)

#### **B. 예제 2**

---


* https://www.kaggle.com/c/titanic/data

**<kaggle의 타이타닉 data>**

  * survived : 생존=1, 죽음=0
  * pclass : 승객 등급. 1등급=1, 2등급=2, 3등급=3
  * sibsp : 함께 탑승한 형제 또는 배우자 수
  * parch : 함께 탑승한 부모 또는 자녀 수
  * ticket : 티켓 번호
  * cabin : 선실 번호
  * embarked : 탑승장소 S=Southhampton, C=Cherbourg, Q=Queenstown

In [ ]:
DF <- fread("https://raw.githubusercontent.com/Jin0331/TA/master/data/titanic/train.csv") %>%
 as_tibble()

In [ ]:
str(DF)

In [ ]:
DF %>% summary()

* 범주형 변수 확인

In [ ]:
DF$Survived %>% unique()

In [ ]:
DF$Pclass %>% unique()

In [ ]:
DF$Sex %>% unique()

In [ ]:
DF$Ticket %>% unique()

In [ ]:
DF$Embarked %>% unique()

In [ ]:
DF <- DF %>% 
 select(-PassengerId, -Name, -Cabin, -Ticket) %>% mutate_at(c("Pclass","Survived","Sex","Embarked"), factor)
str(DF)

* Survived(생존여부)와 Pclass(승객 등급)의 연관관계 조사

In [ ]:
gmodels::CrossTable(DF$Survived, DF$Pclass, 
                    chisq = T, # 카이제곱 검정
                    expected = T, # 기대값(E) 표시
                    prop.r = F, prop.c = F) # 각 셀의 비율표시(F)

* Survived(생존여부)와 Sex(성별)의 연관관계 조사

In [ ]:
gmodels::CrossTable(DF$Survived, DF$Sex, 
                    chisq = T, # 카이제곱 검정
                    expected = T, # 기대값(E) 표시
                    prop.r = F, prop.c = F) # 각 셀의 비율표시(F)

* Survived(생존여부)와 Embarked(탑승 장소)의 연관관계 조사

In [ ]:
gmodels::CrossTable(DF$Survived, DF$Embarked, 
                    chisq = T, # 카이제곱 검정
                    expected = T, # 기대값(E) 표시
                    prop.r = F, prop.c = F) # 각 셀의 비율표시(F)

* Pclass(승객 등급)과 Sex(성별)의 연관관계 조사

In [ ]:
gmodels::CrossTable(DF$Pclass, DF$Sex, 
                    chisq = T, # 카이제곱 검정
                    expected = T, # 기대값(E) 표시
                    prop.r = F, prop.c = F) # 각 셀의 비율표시(F)

* Sex(성별)과 Embarked(탑승 장소)의 연관관계 조사

In [ ]:
gmodels::CrossTable(DF$Sex, DF$Embarked, 
                    chisq = T, # 카이제곱 검정
                    expected = T, # 기대값(E) 표시
                    prop.r = F, prop.c = F) # 각 셀의 비율표시(F)

### **<데이터베이스 연동>**

* 데이터베이스란?
  - 데이터베이스는 체계화된 데이터의 모임이다. 즉, 작성된 목록으로서 여러 응용시스템들의 통합된 정보들을 저장하여 운영할 수 있는 공용데이터들의 묶음이다. 여러 사람들이 공유하고 사용할 목적으로 통합 관리되는 정보의 집합이다. 논리적으로 연관된 하나 이상의 자료의 모음으로 그 내용을 고도로 구조화함으로써 검색과 갱신의 효율화를 꾀한 것이다. 즉, 몇 개의 자료 파일을 조직적으로 통합하여 자료 항목의 중복을 없애고 자료를 구조화하여 기억시켜 놓은 자료의 집합체라고 할 수 있다.

출처: https://neos518.tistory.com/122 [As I've always been]



In [ ]:
#install.packages("tidyverse")
library(tidyverse)

#### **A. SQLite**

https://ehdvudee.tistory.com/23

---


In [ ]:
install.packages(c("DBI", "RSQLite"))

In [ ]:
library(DBI)
library(RSQLite)

* connection 생성

In [ ]:
con <- RSQLite::dbConnect(drv = SQLite(), dbname = "/sqlite/height_weight.sqlite")

In [ ]:
# 현재 디렉토리
getwd()

In [ ]:
dir.create("sqlite")

In [ ]:
list.dirs()

In [ ]:
# sqlite 폴더가 없음. 따라서 생성
con <- RSQLite::dbConnect(drv = RSQLite::SQLite(), dbname = "sqlite/height_weight.sqlite")

In [ ]:
# RSQLite::dbDisconnect(con) # connection 끊기

In [ ]:
list.files("sqlite/")

In [ ]:
dbGetInfo(dbObj = con)

* 테이블 생성하기

```
ID : 사람의 고유한 ID(primary key), integer
HEIGHT : 키(cm), numeric
WEIGHT : 몸무게(kg), numeric

### PRIMARY KEY ??? --> https://jins-dev.tistory.com/entry/RDB-MySQL-%EC%97%90%EC%84%9C%EC%9D%98-Primary-Key-%EB%9E%80
```



In [ ]:
dbListTables(con)

- ``dbSendQuery``는 DB에 쿼리를 보낼 때 사용. 단, 데이터를 가지고 오지는 못함. 

- 또한, ``dbSendQuery``의 경우 ``dbClearResult`` 함수를 이용하여 보낸 쿼리의 처리결과를 제거해주어야 함

In [ ]:
table_query <- dbSendQuery(conn = con, 
                           statement = "CREATE TABLE HW_table(
                                          ID INTEGER PRIMARY KEY,
                                          HEIGHT NUMERIC,
                                          WEIGHT NUMERIC)")

In [ ]:
dbClearResult(table_query)

* 생성된 table 확인

In [ ]:
dbListTables(con)

In [ ]:
dbExistsTable(con, "HW_table") # False 일 경우, 생성이 안 된 것.

In [ ]:
dbListFields(con, "HW_table")

* 생성된 `HW_table`에 데이터 넣기 - 한번에 1행(row)

In [ ]:
row_add_query  <- dbSendQuery(con, 
                                  "INSERT INTO HW_table (ID, HEIGHT, WEIGHT) VALUES (1, 183, 85)")
dbClearResult(row_add_query)

In [ ]:
# error
row_add_query  <- dbSendQuery(con, 
                                  "INSERT INTO HW_table (ID, HEIGHT, WEIGHT) VALUES (1, 182, 67)")
dbClearResult(row_add_query)

In [ ]:
row_add_query  <- dbSendQuery(con, 
                                  "INSERT INTO HW_table (ID, HEIGHT, WEIGHT) VALUES (2, 170, 68)")
dbClearResult(row_add_query)

In [ ]:
row_add_query  <- dbSendQuery(con, 
                                  "INSERT INTO HW_table (ID, HEIGHT, WEIGHT) VALUES (3, 165, 65)")
dbClearResult(row_add_query)

In [ ]:
row_add_query  <- dbSendQuery(con, 
                                  "INSERT INTO HW_table (ID, HEIGHT, WEIGHT) VALUES (5, 202, 100)")
dbClearResult(row_add_query)

* 테이블에서 데이터 가져오기

In [ ]:
dbGetQuery(con, "SELECT * FROM HW_table")

In [ ]:
DF <- dbGetQuery(con, "SELECT * FROM HW_table")
DF %>% str()

In [ ]:
lm(HEIGHT ~ WEIGHT, data = DF) %>% summary()

In [ ]:
dbGetQuery(con, "SELECT ID, HEIGHT FROM HW_table")

In [ ]:
dbGetQuery(con, "SELECT * FROM HW_table WHERE HEIGHT >= 180")

In [ ]:
dbGetQuery(con, "SELECT * FROM HW_table WHERE HEIGHT >= 180 or WEIGHT >= 68")

* 생성된 `HW_table`에 데이터 넣기 - 한번에 여러행

In [ ]:
temp <- tibble(height = sample(150:210, 500, replace = T), weight = sample(45:120, 500, replace = T)) # 복원 추출(중복 허용)
temp %>% show()

In [ ]:
# 해당 Table의 column 명과 동일해야 함. error
dbWriteTable(con, "HW_table", temp, row.names = F, append = T)

In [ ]:
temp <- tibble(HEIGHT = sample(150:210, 500, replace = T), WEIGHT = sample(45:120, 500, replace = T)) # 복원 추출(중복 허용)
temp %>% show()

In [ ]:
# 해당 Table의 column 명과 동일해야 함.
dbWriteTable(con, "HW_table", temp, row.names = F, append = T)

In [ ]:
# ID의 경우 PRIMARY KEY이므로 자동적으로 중복되지 않은 값이 채워짐.
dbGetQuery(con, "SELECT * FROM HW_table")

* 테이블에 있는 데이터 삭제하기(DELETE FROM ~ WHERE)

In [ ]:
query <- dbSendQuery(con, "DELETE FROM HW_table WHERE HEIGHT >= 200")
dbClearResult(query)

In [ ]:
dbGetQuery(con, "SELECT * FROM HW_table WHERE HEIGHT >= 200")

In [ ]:
# 전체 데이터 삭제(table 삭제가 아님!!)
query <- dbSendQuery(con, "DELETE FROM HW_table")
dbClearResult(query)

In [ ]:
dbGetQuery(con, "SELECT * FROM HW_table")

* table 삭제(DROP TABLE)

In [ ]:
query <- dbSendQuery(con, "DROP TABLE HW_table")
dbClearResult(query)

In [ ]:
dbGetQuery(con, "SELECT * FROM HW_table")

In [ ]:
dbListTables(con, "HW_table")

In [ ]:
dbExistsTable(con, "HW_table")

* connection 종료

In [ ]:
dbDisconnect(con)

* SQLite의 이식성(!?)

In [ ]:
# Lab 9 참고
#https://archive.ics.uci.edu/ml/datasets.php?format=&task=reg&att=&area=&numAtt=&numIns=&type=&sort=dateDown&view=table
#https://datascience.stackexchange.com/questions/8922/removing-strings-after-a-certain-character-in-a-given-text
#https://stackoverflow.com/questions/5992082/how-to-remove-all-whitespace-from-a-string

Sys.setlocale('LC_ALL','C') # 이부분 무시해도 됨.
DF <- fread("https://raw.githubusercontent.com/Jin0331/TA/master/data/SeoulBikeData.csv", sep = ",", stringsAsFactors = F) %>%
 as_tibble()

colnames(DF) <- colnames(DF) %>% lapply(X = ., FUN = function(value){
  stringr::str_remove(string = value, pattern = "\\(.*") %>%
  stringr::str_replace_all(string = ., pattern = "[[:space:]]", replace = "_")
}) %>% unlist()

DF %>% str()

In [ ]:
con <- RSQLite::dbConnect(drv = RSQLite::SQLite(), dbname = "sqlite/seoul_bike.sqlite")

In [ ]:
dbWriteTable(con, "seoul_bike", DF, row.names = F, append = F)

In [ ]:
dbListTables(con)

In [ ]:
dbListFields(con, "seoul_bike")

In [ ]:
dbGetQuery(con, "SELECT * FROM seoul_bike") %>%
 as_tibble() %>% str()

In [ ]:
dbDisconnect(con)

#### **B. dbplyr**

https://dbplyr.tidyverse.org/index.html

https://dbplyr.tidyverse.org/articles/sql-translation.html

```
dbplyr is the database backend for dplyr. 
It allows you to use remote database tables as if they are in-memory data frames by automatically converting dplyr code into SQL.
```

---


In [ ]:
con <- RSQLite::dbConnect(drv = RSQLite::SQLite(), dbname = "sqlite/seoul_bike.sqlite")

* copy_to를 local 데이터를 database에 복제

In [ ]:
dbListTables(con)

In [ ]:
dplyr::copy_to(con, iris)

In [ ]:
dbListTables(con)

* tbl을 이용한 database의 table 사용(dataframe)

In [ ]:
iris_2 <- tbl(con, "iris")
iris_2

In [ ]:
iris %>% show()

In [ ]:
seoul_bike_2 <- tbl(con, "seoul_bike")
seoul_bike_2

* dplyr 문법을 이용한 SQL 번역

In [ ]:
seoul_bike_2 %>% select(Date, Hour)

In [ ]:
seoul_bike_2 %>% filter(Humidity >= 40)

In [ ]:
seoul_bike_2 %>% group_by(Date) %>% summarise(daily_humidity = mean(Humidity, na.rm = T))

* Show Query

In [ ]:
daily_h <- seoul_bike_2 %>% filter(Humidity >= 40) %>% 
 group_by(Date) %>% summarise(daily_humidity = mean(Humidity, na.rm = T))

In [ ]:
show_query(daily_h)

In [ ]:
explain(daily_h)

* Collect Data

```
Now, some interesting facts. When working with databases, dplyr never pulls data into R unless you explicitly ask for it. 
In the previous example, dplyr will not do anything until you ask for the mileages data. 

It generates the SQL and only pulls down a few rows when you try to print mileages. So how do we pull all the data and store it for further analysis? 
collect() will pull all the data and store it in a tibble and you can use it for any further analysis.
```

In [ ]:
daily_h %>% collect() %>% show()

#### **C. Google Cloud Platform(GCP)를 이용한 DB 서버 생성 및 이용**

https://blog-rogan.tistory.com/entry/Cloud-Platform%EC%9D%B4%EB%9E%80

https://cloud.google.com/free/docs/gcp-free-tier?hl=ko

---


In [ ]:
system("sudo apt-get install libmysql++-dev")

In [ ]:
install.packages("RMariaDB")

In [ ]:
#dbDisconnect(con)

In [ ]:
con <- dbConnect(RMariaDB::MariaDB(), user = "root", password = "spark123!",
                          host = "34.123.12.131", dbname = "test")

* 예제 데이터

In [ ]:
#install.packages("nycflights13")
nycflights13::flights %>% str()

In [ ]:
copy_to(con, nycflights13::flights, "flights")

* 예제 데이터

In [ ]:
Sys.setlocale('LC_ALL','C') # 이부분 무시해도 됨.
DF <- fread("https://raw.githubusercontent.com/Jin0331/TA/master/data/SeoulBikeData.csv", sep = ",", stringsAsFactors = F) %>%
 as_tibble()

colnames(DF) <- colnames(DF) %>% lapply(X = ., FUN = function(value){
  stringr::str_remove(string = value, pattern = "\\(.*") %>%
  stringr::str_replace_all(string = ., pattern = "[[:space:]]", replace = "_")
}) %>% unlist()

In [ ]:
copy_to(con, DF, "seoul_bike")

- - -

## **실습 과제 <11주차 과제 - 100점>**

### **(1) 아래의 조건에 맞는 ```학번_이름.sqlite 파일```을 생성한 후, 한림SmartLEAD에 제출할 것(ex.D20514_이진우.sqlite)**
### **(2) 제출 데드라인 : 2020.11.9(월요일) ~ 2020.11.15(일요일) 23:59 까지**


- - -


### **1. SQLite를 이용하여 학번_이름.sqlite 데이터베이스를 생성**

### **2. dbplyr를 이용하여 학번_이름(영어)인 테이블 생성 (ex. D20514_jinoo)**

```
<테이블의 구조>

ID : 자신의 학번
GRADE : 자신의 학년
NAME : 자신의 이름
RESIDENCE : 자신의 거주 지역
```


### **3. dbplyr를 이용하여 각 열에 맞는 데이터 삽입(1행만)**



* 예시 코드

In [ ]:
install.packages(c("DBI", "RSQLite", "tidyverse", "dbplyr"))
library(DBI)
library(RSQLite)
library(tidyverse)
library(dbplyr)

In [27]:
getwd()
dir.create("test")
list.dirs()

[1] "/content"

[1] "."                         "./.config"                
[3] "./.config/configurations"  "./.config/logs"           
[5] "./.config/logs/2020.10.28" "./.ipynb_checkpoints"     
[7] "./sample_data"             "./test"

In [28]:
con <- RSQLite::dbConnect(drv = RSQLite::SQLite(), dbname = "test/D20514_이진우.sqlite") # 자신의 이름과 학번으로 설정할 것

* local dataframe 생성

In [29]:
DF <- tibble(ID = c("D20514"), GRADE = c("1"), NAME = c("이진우"), RESIDENCE = c("서울 성북구/강원 춘천.."))
DF %>% show()

# A tibble: 1 x 4
  ID     GRADE NAME   RESIDENCE              
  <chr>  <chr> <chr>  <chr>                  
1 D20514 1     이진우 서울 성북구/강원 춘천..


* copy_to를 이용하여 local dataframe을 SQLite의 테이블로 생성

In [30]:
dplyr::copy_to(con, DF, "D20514_JinwooLee", temporary = F) # connection, 생성한 dataframe, 테이블 명, 일시적인 테이블 해제

* 생성한 테이블 확인

In [31]:
dbListTables(con)

[1] "D20514_JinwooLee" "sqlite_stat1"     "sqlite_stat4"

In [32]:
tbl(con, "D20514_JinwooLee") %>% collect() %>% show()

# A tibble: 1 x 4
  ID     GRADE NAME   RESIDENCE              
  <chr>  <chr> <chr>  <chr>                  
1 D20514 1     이진우 서울 성북구/강원 춘천..


In [33]:
dbDisconnect(con)